## Data Loading

In [1]:
import numpy as np
import pandas as pd
churn = pd.read_csv('dataset.csv', sep=',')

## Data Analysis

In [2]:
data_size = churn.shape
print(data_size)

(3333, 21)


In [3]:
churn_col_names = list(churn.columns)
print(churn_col_names)

['state', 'account length', 'area code', 'phone number', 'international plan', 'voice mail plan', 'number vmail messages', 'total day minutes', 'total day calls', 'total day charge', 'total eve minutes', 'total eve calls', 'total eve charge', 'total night minutes', 'total night calls', 'total night charge', 'total intl minutes', 'total intl calls', 'total intl charge', 'customer service calls', 'churn']


In [4]:
print(churn.describe())

       account length    area code  number vmail messages  total day minutes  \
count     3333.000000  3333.000000            3333.000000        3333.000000   
mean       101.064806   437.182418               8.099010         179.775098   
std         39.822106    42.371290              13.688365          54.467389   
min          1.000000   408.000000               0.000000           0.000000   
25%         74.000000   408.000000               0.000000         143.700000   
50%        101.000000   415.000000               0.000000         179.400000   
75%        127.000000   510.000000              20.000000         216.400000   
max        243.000000   510.000000              51.000000         350.800000   

       total day calls  total day charge  total eve minutes  total eve calls  \
count      3333.000000       3333.000000        3333.000000      3333.000000   
mean        100.435644         30.562307         200.980348       100.114311   
std          20.069084          9.25943

In [5]:
print(churn.head())

  state  account length  area code phone number international plan  \
0    KS             128        415     382-4657                 no   
1    OH             107        415     371-7191                 no   
2    NJ             137        415     358-1921                 no   
3    OH              84        408     375-9999                yes   
4    OK              75        415     330-6626                yes   

  voice mail plan  number vmail messages  total day minutes  total day calls  \
0             yes                     25              265.1              110   
1             yes                     26              161.6              123   
2              no                      0              243.4              114   
3              no                      0              299.4               71   
4              no                      0              166.7              113   

   total day charge  ...  total eve calls  total eve charge  \
0             45.07  ...           

## Target Identification

In [6]:
churn_target = churn['churn']
print(churn_target.unique())

[False  True]


## Feature Identification

In [9]:
#Phone number is unique and might not influence prediction
#churn is the target variable and should be dropped

cols_to_drop = ['phone number', 'churn']
churn_feature = churn.drop(cols_to_drop, axis=1)
print(churn_feature.head())

  state  account length  area code international plan voice mail plan  \
0    KS             128        415                 no             yes   
1    OH             107        415                 no             yes   
2    NJ             137        415                 no              no   
3    OH              84        408                yes              no   
4    OK              75        415                yes              no   

   number vmail messages  total day minutes  total day calls  \
0                     25              265.1              110   
1                     26              161.6              123   
2                      0              243.4              114   
3                      0              299.4               71   
4                      0              166.7              113   

   total day charge  total eve minutes  total eve calls  total eve charge  \
0             45.07              197.4               99             16.78   
1             27.47   

## Data Preprocessing

In [10]:
#Categorical Data

churn_categorical = churn.select_dtypes(include=[object])
print(churn_categorical.head(2))

  state phone number international plan voice mail plan
0    KS     382-4657                 no             yes
1    OH     371-7191                 no             yes


In [11]:
#Changing yes/no values to boolean

yes_no_cols = ['international plan', 'voice mail plan']
churn_feature[yes_no_cols] = churn_feature[yes_no_cols] == 'yes'
print(churn_feature.head(2))

  state  account length  area code  international plan  voice mail plan  \
0    KS             128        415               False             True   
1    OH             107        415               False             True   

   number vmail messages  total day minutes  total day calls  \
0                     25              265.1              110   
1                     26              161.6              123   

   total day charge  total eve minutes  total eve calls  total eve charge  \
0             45.07              197.4               99             16.78   
1             27.47              195.5              103             16.62   

   total night minutes  total night calls  total night charge  \
0                244.7                 91               11.01   
1                254.4                103               11.45   

   total intl minutes  total intl calls  total intl charge  \
0                10.0                 3                2.7   
1                13.7        

In [12]:
#Label Encoding

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
churn_feature['area code'] = label_encoder.fit_transform(churn_feature['area code'])
print(churn_feature.head(2))

  state  account length  area code  international plan  voice mail plan  \
0    KS             128          1               False             True   
1    OH             107          1               False             True   

   number vmail messages  total day minutes  total day calls  \
0                     25              265.1              110   
1                     26              161.6              123   

   total day charge  total eve minutes  total eve calls  total eve charge  \
0             45.07              197.4               99             16.78   
1             27.47              195.5              103             16.62   

   total night minutes  total night calls  total night charge  \
0                244.7                 91               11.01   
1                254.4                103               11.45   

   total intl minutes  total intl calls  total intl charge  \
0                10.0                 3                2.7   
1                13.7        

In [22]:
#One Hot Encoding

print('Churn data size before One Hot Encoding :', churn_feature.shape)
print('No. of unique states :', len(churn_feature['state'].unique()))

#Give the feature and columns to One Hot Encode in 'columns' and column rename prefix in 'prefix'
churn_dumm = pd.get_dummies(churn_feature, columns=['state'], prefix=['state'])
print('Churn data size after One Hot Encoding :', churn_dumm.shape)

import numpy as np  #Converting to numpy matrix
churn_matrix = churn_dumm.values.astype(np.float64)


Churn data size before One Hot Encoding : (3333, 19)
No. of unique states : 51
Churn data size after One Hot Encoding : (3333, 69)


In [23]:
#Imputing Missing values

from sklearn.impute import SimpleImputer
#Missing values replaced by mean
imp = SimpleImputer(missing_values=np.nan, strategy='mean', fill_value=None, copy=True)
#Fit to data, then transform it
churn_matrix = imp.fit_transform(churn_matrix)

In [24]:
#Standardization

from sklearn.preprocessing import StandardScaler
#Standardize the data by removing the mean and scaling to unit variance
scaler = StandardScaler()
#Fit to data, then transform it
churn_matrix = scaler.fit_transform(churn_matrix)

## Train and Test Data

In [25]:
seed = 7   #To generate same sequence of random numbers
from sklearn.model_selection import train_test_split
#Splitting the data for training and testing (90% train, 10% test)
train_data, test_data, train_label, test_label = train_test_split(churn_matrix, churn_target, test_size=0.1, random_state=seed)

## Decision Tree Classifier

In [26]:
from sklearn.tree import DecisionTreeClassifier
#Initializing Decision Tree Classifier
classifier = DecisionTreeClassifier(random_state=seed)

#Model training
classifier = classifier.fit(train_data, train_label)

#After being fitted, the model can then be used to predict the output
churn_predicted_target = classifier.predict(test_data)

#Evaluating the classifier
score = classifier.score(test_data, test_label)
print('Decision Tree Classifier :', score)

Decision Tree Classifier : 0.9101796407185628


## Naive Bayes Classifier

In [27]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier = classifier.fit(train_data, train_label)
churn_predicted_target = classifier.predict(test_data)
score = classifier.score(test_data, test_label)
print('Naive Bayes :', score)

Naive Bayes : 0.5748502994011976


## Stochastic Gradient Descent Classifier

In [28]:
from sklearn.linear_model import SGDClassifier
classifier =  SGDClassifier(loss='modified_huber', shuffle=True,random_state=seed)
classifier = classifier.fit(train_data, train_label)
churn_predicted_target = classifier.predict(test_data)
score = classifier.score(test_data, test_label)
print('SGD classifier :', score)

SGD classifier : 0.781437125748503


## Support Vector Machine

In [29]:
from sklearn.svm import SVC
classifier = SVC(kernel="linear", C=0.025,random_state=seed)
classifier = classifier.fit(train_data, train_label)
churn_predicted_target = classifier.predict(test_data)
score = classifier.score(test_data, test_label)
print('SVM Classifier :', score)

SVM Classifier : 0.8473053892215568


## Random Forest Classifier

In [30]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=10, random_state=seed)
classifier = classifier.fit(train_data, train_label)
churn_predicted_target = classifier.predict(test_data)
score = classifier.score(test_data, test_label)
print('Random Forest Classifier :', score)

Random Forest Classifier : 0.8682634730538922


## Model Tuning

In [31]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=5, n_estimators=15, max_features=60,random_state=seed)
classifier = classifier.fit(train_data, train_label)
score = classifier.score(test_data, test_label)
print('Random Forest classification after model tuning :', score)

Random Forest classification after model tuning : 0.937125748502994


## Stratified Shuffle Split

In [32]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1,test_size=0.1, random_state=7)
sss.get_n_splits(churn_matrix,churn_target)
print(sss)

StratifiedShuffleSplit(n_splits=1, random_state=7, test_size=0.1,
            train_size=None)


In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
classifiers = [
    DecisionTreeClassifier(),
    GaussianNB(),
    SGDClassifier(loss='modified_huber', shuffle=True),
    SVC(kernel="linear", C=0.025),
    KNeighborsClassifier(),
    OneVsRestClassifier(svm.LinearSVC()),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=10),
    AdaBoostClassifier(),
   ]
for clf in classifiers:
    score=0
    for train_index, test_index in sss.split(churn_matrix,churn_target):
        X_train, X_test = churn_matrix[train_index], churn_matrix[test_index]
        y_train, y_test = churn_target[train_index], churn_target[test_index]
        clf.fit(X_train, y_train)
        score=score+clf.score(X_test, y_test)
    print(score)

0.8922155688622755
0.6137724550898204
0.7784431137724551
0.8562874251497006
0.8652694610778443


C:\Users\chakr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\chakr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\chakr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


0.8652694610778443
0.8922155688622755
0.8832335329341318


## Classification Accuracy

In [34]:
from sklearn.metrics import accuracy_score
print('Accuracy Score',accuracy_score(test_label,churn_predicted_target))  

Accuracy Score 0.8682634730538922


## Confusion Matrix

In [36]:
from sklearn.metrics import confusion_matrix
print('Confusion Matrix\n', confusion_matrix(test_label, churn_predicted_target))

Confusion Matrix
 [[282   1]
 [ 43   8]]


## Classification Report

In [37]:
from sklearn.metrics import classification_report
target_names = ['False.', 'True.']
print(classification_report(test_label, churn_predicted_target, target_names=target_names))

              precision    recall  f1-score   support

      False.       0.87      1.00      0.93       283
       True.       0.89      0.16      0.27        51

    accuracy                           0.87       334
   macro avg       0.88      0.58      0.60       334
weighted avg       0.87      0.87      0.83       334

